In [1]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [ ]:
from transformers import Trainer, BertForSequenceClassification, BertTokenizer, EarlyStoppingCallback
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch
import base
import os 
import copy

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
base.reset_seed()

In [3]:
DATASET = "trec"

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [5]:
train = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

train_aug = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")

In [6]:
tokenizer = BertTokenizer.from_pretrained("ndavid/autotrain-trec-fine-bert-739422530")

In [7]:
train = train.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = train_aug.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

In [ ]:
train_data_gpu = copy.deepcopy(train_data)
train_data_gpu.set_format(type="torch", columns=["input_ids", "attention_mask"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_data)
train_data_cpu.set_format(type="torch", columns=["input_ids","attention_mask"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

In [8]:
base.reset_seed()

In [9]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [154]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_fine", logging_dir=f"~/logs/{DATASET}/bert-base_fine", lr=0.0012, weight_decay=.01, warmup_steps=4, batch_size=128, epochs=20)

In [155]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [156]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.072300,2.295601,0.505041,0.161854,0.165593,0.143170
2,1.831300,1.538879,0.660862,0.273265,0.284365,0.253168
3,1.158200,1.244950,0.719523,0.345682,0.353773,0.332803
4,0.800200,1.237962,0.719523,0.379431,0.396922,0.364396
5,0.567800,1.113162,0.754354,0.489907,0.464247,0.454477
6,0.397900,1.110101,0.758937,0.558650,0.506712,0.516080
7,0.271600,1.075222,0.766269,0.566544,0.548188,0.540473
8,0.202800,1.189122,0.757104,0.603306,0.578967,0.576549
9,0.149400,1.138134,0.762603,0.623862,0.585633,0.588227
10,0.114900,1.139649,0.777269,0.694329,0.649343,0.651218


TrainOutput(global_step=525, training_loss=0.5915996124630882, metrics={'train_runtime': 79.107, 'train_samples_per_second': 1102.557, 'train_steps_per_second': 8.849, 'total_flos': 49425716214000.0, 'train_loss': 0.5915996124630882, 'epoch': 15.0})

In [157]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [158]:
trainer.evaluate(test)

{'eval_loss': 1.0590753555297852,
 'eval_accuracy': 0.796,
 'eval_precision': 0.6639781101834673,
 'eval_recall': 0.6490349427918719,
 'eval_f1': 0.6378382138608072,
 'eval_runtime': 3.4501,
 'eval_samples_per_second': 144.924,
 'eval_steps_per_second': 1.159,
 'epoch': 15.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-base_fine.pth")

In [138]:
base.reset_seed()

In [139]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [140]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_fine", logging_dir=f"~/logs/{DATASET}/bert-distill_fine", remove_unused_columns=False, lr=0.0015, weight_decay=.003, warmup_steps=4, batch_size=128, epochs=20, temp=6, lambda_param=.4)

In [141]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [142]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.141800,1.637039,0.518790,0.153312,0.173378,0.142616
2,1.307400,1.139175,0.651696,0.246957,0.261602,0.236382
3,0.898400,0.960543,0.707608,0.288119,0.320861,0.298602
4,0.657900,0.886360,0.738772,0.383868,0.380070,0.367709
5,0.496900,0.833152,0.758020,0.490537,0.451600,0.449750
6,0.392300,0.811514,0.770852,0.537126,0.479521,0.490648
7,0.291200,0.806628,0.770852,0.553484,0.524728,0.524161
8,0.233000,0.808993,0.778185,0.601631,0.569304,0.569710
9,0.186300,0.803415,0.778185,0.621739,0.557800,0.573217
10,0.162400,0.818579,0.779102,0.644498,0.615526,0.611536


TrainOutput(global_step=560, training_loss=0.4668802525315966, metrics={'train_runtime': 86.4269, 'train_samples_per_second': 1009.177, 'train_steps_per_second': 8.099, 'total_flos': 52720763961600.0, 'train_loss': 0.4668802525315966, 'epoch': 16.0})

In [143]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [144]:
trainer.evaluate(test)

{'eval_loss': 0.8118124008178711,
 'eval_accuracy': 0.766,
 'eval_precision': 0.6861004212477909,
 'eval_recall': 0.6483048743655664,
 'eval_f1': 0.6356455735630214,
 'eval_runtime': 3.3471,
 'eval_samples_per_second': 149.384,
 'eval_steps_per_second': 1.195,
 'epoch': 16.0}

In [44]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distil_fine.pth")

In [35]:
base.reset_seed()

In [36]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base-aug_fine", logging_dir=f"~/logs/{DATASET}/bert-base-aug_fine", lr=0.00018, warmup_steps=25,  epochs=20)

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.727300,1.114826,0.761687,0.448947,0.448363,0.437642
2,0.445500,0.992170,0.774519,0.606985,0.551132,0.554259
3,0.179600,1.029273,0.786434,0.642513,0.599345,0.608961
4,0.088800,1.056720,0.804766,0.777675,0.707703,0.726482
5,0.052800,1.124146,0.794684,0.748868,0.688627,0.703290
6,0.033800,1.161935,0.800183,0.778954,0.710943,0.727378
7,0.025800,1.246127,0.790101,0.769173,0.710310,0.722345
8,0.019700,1.226771,0.790101,0.774692,0.712997,0.726748
9,0.015300,1.295860,0.794684,0.803265,0.706617,0.733698
10,0.011900,1.372799,0.791017,0.776010,0.709725,0.721296


TrainOutput(global_step=6336, training_loss=0.21836013053402756, metrics={'train_runtime': 222.4221, 'train_samples_per_second': 6076.373, 'train_steps_per_second': 47.477, 'total_flos': 612702077299200.0, 'train_loss': 0.21836013053402756, 'epoch': 12.0})

In [40]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [41]:
trainer.evaluate(test)

{'eval_loss': 1.2343593835830688,
 'eval_accuracy': 0.784,
 'eval_precision': 0.6889687628677962,
 'eval_recall': 0.6815071194775028,
 'eval_f1': 0.6608612287748105,
 'eval_runtime': 3.7143,
 'eval_samples_per_second': 134.615,
 'eval_steps_per_second': 1.077,
 'epoch': 12.0}

In [26]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-base-aug_fine.pth")

In [27]:
base.reset_seed()

In [28]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill-aug_fine", logging_dir=f"~/logs/{DATASET}/bert-distill-aug_fine", remove_unused_columns=False, lr=0.00047, weight_decay=.007, warmup_steps=15, epochs=20, temp=4, lambda_param=.8)

In [30]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.515600,0.471725,0.779102,0.549351,0.514425,0.513555
2,0.112800,0.459124,0.786434,0.618439,0.601509,0.598734
3,0.080800,0.475293,0.780935,0.719573,0.626858,0.651858
4,0.070900,0.463248,0.796517,0.736075,0.654855,0.680492
5,0.066100,0.456250,0.793767,0.748872,0.664825,0.691250
6,0.062200,0.470117,0.795600,0.748254,0.668805,0.693948
7,0.060000,0.481648,0.785518,0.760266,0.672599,0.700890
8,0.058700,0.460094,0.796517,0.749415,0.693793,0.709451
9,0.057200,0.471035,0.797434,0.785497,0.692480,0.720783
10,0.056100,0.473911,0.791934,0.774716,0.700257,0.722220


TrainOutput(global_step=10560, training_loss=0.08340635299682617, metrics={'train_runtime': 363.5742, 'train_samples_per_second': 3717.315, 'train_steps_per_second': 29.045, 'total_flos': 1021170128832000.0, 'train_loss': 0.08340635299682617, 'epoch': 20.0})

In [32]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [33]:
trainer.evaluate(test)

{'eval_loss': 0.41090407967567444,
 'eval_accuracy': 0.806,
 'eval_precision': 0.6752235121669131,
 'eval_recall': 0.6893776846211949,
 'eval_f1': 0.6622485946860568,
 'eval_runtime': 3.5522,
 'eval_samples_per_second': 140.757,
 'eval_steps_per_second': 1.126,
 'epoch': 20.0}

In [34]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distil-aug_fine.pth")

In [ ]:
base.count_parameters(student_model)

In [ ]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

In [ ]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())